<a href="https://colab.research.google.com/github/8johan/ML-PROJ/blob/main/copie3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch 
from __future__ import print_function, division
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as fn
import torch.utils.data
import torch
def double_conv(data , intc,outc):
  convo=nn.Sequential(
            nn.Conv2d(intc, outc, kernel_size=3, stride=1, padding=0, bias=True),
            #nn.BatchNorm2d(outc),
            #nn.ReLU(inplace=True),
            nn.Conv2d(outc, outc, kernel_size=3, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(outc),
            nn.ReLU(inplace=True))
  x=convo(data)
  return x
def upconv(data , intc, outc):
  upco=nn.ConvTranspose2d(intc, outc, kernel_size=2, stride=2, padding=0)
  x=upco(data)
  return x 
def crop_copy(img,x):
  croped_size=x.size()[2]
  crop = fn.center_crop(img, output_size=[croped_size])
  y=torch.cat((x, crop), dim=1)
  return y





In [2]:
class countarctive_path(nn.Module):
  def __init__(self) :
    
    super().__init__()
    self.l=[]
    self.maxpoll=nn.MaxPool2d(kernel_size=2, stride=2)
  def forward(self,data):
    x=double_conv(data,3,64)
    self.l.append(x)
    x=self.maxpoll(x)
   
    print(x.size())
    x=double_conv(x,64,128)
    self.l.append(x)
    x=self.maxpoll(x)
 
    print(x.size())
    x=double_conv(x,128,256)
    self.l.append(x)
    x=self.maxpoll(x)
    
    print(x.size())
    x=double_conv(x,256,512)
    self.l.append(x)
    x=self.maxpoll(x)
  
    print(x.size())
    x=double_conv(x,512,1024)
  
    
  
    return x

  




In [3]:
class expansive_path(nn.Module):
  def __init(self,x):
    super().__init__()
    self.l=x

  def forward(self , data ,l):
    x=upconv(data , 1024 ,512)
    x=crop_copy(l[3],x)
    x=double_conv(x ,1024 ,512)
    x=upconv(x, 512 ,256)
    x=crop_copy(l[2],x)
    x=double_conv(x ,512 ,256)
    x=upconv(x , 256,128)
    x=crop_copy(l[1],x)
    x=double_conv(x ,256 ,128)
    x=upconv(x , 128 ,64)
    x=crop_copy(l[0],x)
    x=double_conv(x ,128 ,64)
    conv1=nn.Conv2d(64, 2, kernel_size=1, stride=1, padding=0, bias=True)
    x=conv1(x)
    print(x.size())
    return x 
   
    




In [4]:
class unet(nn.Module):
  def __init__(self) :
    super().__init__()
    self.countratpath=countarctive_path()
    self.expansivepath=expansive_path()
  def forward(self,data):
    x=self.countratpath.forward(data)
    l=self.countratpath.l
    x=self.expansivepath.forward(x,l)
    
    return x


In [7]:
inputs = torch.randn((2, 3, 572, 572))
modul=unet()
y=modul.forward(inputs)

torch.Size([2, 64, 284, 284])
torch.Size([2, 128, 140, 140])
torch.Size([2, 256, 68, 68])
torch.Size([2, 512, 32, 32])
torch.Size([2, 2, 388, 388])
